# **<center><font style="color:rgb(100,109,254)">Module 2: Real-Time Controllable Face Makeup</font> </center>**

<img src='https://drive.google.com/uc?export=download&id=1ERLn4WIEsSGnAO_4LDT8R8FTf6ac3f5n'>

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to Face Landmark Detection Theory*

- *Lesson 2: Create a Face Landmarks Detector*

- ***Lesson 3:* Build a Face Part Selector** *(This Tutorial)*

- *Lesson 4: Build a Virtual Face Makeup Application*

- *Lesson 5: Build the Final Application*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [1]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
from importlib.metadata import version
from previous_lesson import detectFacialLandmarks, detectHandsLandmarks, countFingers
print(f"Mediapipe version: {version('mediapipe')}, it should be 0.8.9.1")


# Additional comments:
#       - Use the previous functions on detection of hands, counting fingers
#         and detection of facial features. The hands will be used as the controller

Mediapipe version: 0.8.10.1, it should be 0.8.9.1


## **<font style="color:rgb(134,19,348)">Initializations</font>**

After that, in this step, we will perform all the initializations required to build the Face Parts Selector. Note that we want to select the Face Parts by touching them with our hands.

### **<font style="color:rgb(134,19,348)">Face Landmarks Detection Model</font>**

So first, we will have to initialize the **`mp.solutions.face_mesh`** class and then set up the **`mp.solutions.face_mesh.FaceMesh()`** function with appropriate arguments, as we had done in the previous lesson. We will only be working with videos in this lesson, so we will only set up the **`mp.solutions.face_mesh.FaceMesh()`** function one time with **`static_image_mode`** to `False`.

In [2]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Set up the face mesh function with appropriate arguments.
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True,
                                  min_detection_confidence=0.8, min_tracking_confidence=0.5)

# Additional comments:
#       - Since we will not be detecting static images, we will
#         only get an instance of the face mesh, and set the 
#         static image mode to False.

### **<font style="color:rgb(134,19,348)">Hands Landmarks Detection Model</font>**

As mentioned we want to select the face parts with hands, so we will also need the **Mediapipe's Hands solution** (explained in detail in the previous module) as well, to get the exact location and gesture of the hands, so now we will have to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments. 

In [3]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands function with appropriate arguments.
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2,
                       min_detection_confidence=0.8, min_tracking_confidence=0.8)

# Additional comments:
#       - The hands mesh will be used to determine whether
#         the tip of the pointing finger of the right hand
#         is in the region of interest (e.g. Right cheeks, eyes, lips)

### **<font style="color:rgb(134,19,348)">Face Parts Indexes</font>**

Now as we already know, for each face, **Mediapipe's Face Mesh solution** returns a list of **four hundred sixty-eight** facial landmarks that represent important regions of the face e.g, eyes, nose, lips, etc and to build a face parts selector we need to figure out a way to isolate the landmarks of the face parts from the face mesh returned by the solution. Well, luckily similar to the Mediapipe's Hand solution, the Face Mesh solution also always returns these landmarks in the same sequence meaning that for every face the landmark representing the nose tip will be at the index 1 in the list. You can find the indexes for the face parts, that you want to isolate in the image below. 

<center>

<img src='https://drive.google.com/uc?export=download&id=1wOuAzZfXKJ1OZhE94PqGbv4ab1fPEZJy' width ="600"> 
    
<br><a href="https://developers.google.com/ar/develop/augmented-faces">Image Source</a>

</center>
    
Now we will initialize a few lists containing the indexes of landmarks of the face parts (lips, right eye, and right cheek) that we want to select. Mediapipe also provides some frozenset objects (as attributes of the **`mp.solutions.face_mesh`** class) that contain indexes of some face parts landmark coordinates. 

* **`mp_face_mesh.FACEMESH_FACE_OVAL`** contains indexes of face outline.
* **`mp_face_mesh.FACEMESH_LIPS`** contains indexes of lips.
* **`mp_face_mesh.FACEMESH_LEFT_EYE`** contains indexes of left eye.
* **`mp_face_mesh.FACEMESH_RIGHT_EYE`** contains indexes of right eye.
* **`mp_face_mesh.FACEMESH_LEFT_EYEBROW`** contains indexes of left eyebrow.
* **`mp_face_mesh.FACEMESH_RIGHT_EYEBROW`** contains indexes of right eyebrow.

But these frozenset objects contain the indexes of all the landmarks of the face parts and we only need the landmarks indexes of the outlines of these face parts so that's why we are going with manually defining the indexes lists, this approach no doubt is more time consuming as we have to manually find the indexes but gives much more control.

In [4]:
# Initialize a list to store the indexes of the upper lips outer outline landmarks.
lips_upper_outer_ids = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]

# Initialize a list to store the indexes of the lower lips outer outline landmarks.
lips_lower_outer_ids = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291]

# Initialize a list to store the indexes of the upper part of the right eye outline landmarks.
right_eye_upper_ids = [342, 445, 444, 443, 442, 441, 413]

# Initialize a list to store the indexes of the lower part of the right eye outline landmarks.
right_eye_lower_ids = [463, 341, 256, 252, 253, 254, 339, 255, 359]

# Initialize a list to store the indexes of the right cheek outline landmarks.
right_cheek_landmarks_ids = [379, 365, 397, 288, 361, 323, 454, 356, 372, 346, 280, 425, 432, 430]


# For homework, eyebrow landmarks
# Initialize a list to store the indexes of the upper part of the right eye outline landmarks.
# With this upper ids, the eyebrow fits perfectly.
# However, when i'm pointing to the eyebrow, it doesn't detect
#right_eyebrow_upper_ids = [293, 334, 296, 336]

right_eyebrow_upper_ids = [293, 333, 299, 337, 336]
right_eyebrow_lower_ids = [285, 295, 282, 283, 276]

left_eyebrow_upper_ids = [63, 104, 69, 108, 151, 9, 8]
left_eyebrow_lower_ids = [55, 65, 52, 53, 46]

# Additional comments:
#       - This will contain the index of the landmarks
#         that is in our targetted region of interest

## **<font style="color:rgb(134,19,348)">Create a Face Part Selection Function</font>**

Now we will create a function **`selectFacePart()`** that will allow the user to select **Face, Eyes, and Lips** by just touching these face parts in real-time with the index fingertip while making the `INDEX POINTING UP` hand gesture (☝️). We will utilize the **`countFingers()`** function from the previous module, to count the fingers up and get the tips landmarks, and then we will use the OpenCV's [**`cv2.pointPolygonTest()`**](https://docs.opencv.org/3.4/d3/dc0/group__imgproc__shape.html#ga1a539e8db2135af2566103705d7a5722) function to check whether the finger is touching a face part. We will also use the [**`cv2.drawContours()`**](https://docs.opencv.org/3.4/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc) function to highlight the selectable face parts.

In [5]:
def selectFacePart(image, face_landmarks, hands_results, hand_label='RIGHT'):
    '''
    This function will allow the user to select face parts utilizing hand gestures.
    Args:
        image:          The image/frame of the user with his index finger pointing towards a face part to select.
        face_landmarks: An array containing the face landmarks (x and y coordinates) of the face in the image.
        hands_results:  The output of the hands landmarks detection performed on the image. 
        hand_label:     The label of the hand i.e. left or right, of which the gesture is required to be recognized. 
    Returns:
        output_image:       A copy of the input image with transparent contours drawn, highlighting the selectable face parts. 
        selected_face_part: The name of the face part selected by the user in the image/frame.
    '''
    
    # Initialize a variable to store the selected face part.
    selected_face_part = None
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Initialize a list to store the lips landmarks.
    lips_landmarks = []
    
    # Initialize a list to store the right eye landmarks.
    right_eye_landmarks = []

    # For homework, eyebrow landmarks
    right_eyebrow_landmarks = []
    left_eyebrow_landmarks = []
    
    # Initialize a list to store the right cheek landmarks.
    right_cheek_landmarks = []
    
    # Get the height and width of the image.
    height, width, _ = image.shape

    # Get the count of fingers up, fingers statuses, and tips landmarks of the detected hand(s).
    # I have modified this countFingers() function from previous module to ignore the thumbs count logic, if consider_thumbs
    # is False, to remove the limatation of always having to face the palm of hand towards the camera to get correct results.
    count, fingers_statuses, fingers_tips_position = countFingers(image, hands_results, consider_thumbs=False,
                                                                  draw=False, display=False)
    
    # Check if the number of the fingers up of the selector hand is 1 and the finger that is up, is the index finger.
    # And the number of the fingers up, of the opposite hand is 0.
    if count[hand_label] == 1 and fingers_statuses[hand_label+'_INDEX'] \
    and count['LEFT' if hand_label=='RIGHT' else 'RIGHT'] == 0:
        
        # Get the x and y coordinates of the tip landmark of the index finger of the selector hand. 
        index_x, index_y = fingers_tips_position[hand_label]['INDEX']
        
        # Lips Selection part.
        ####################################################################################################################
        
        # Iterate over the indexes of the upper and lower lips outline.
        for index in lips_upper_outer_ids+lips_lower_outer_ids:
            
            # Get the landmark at the index we are iterating upon,
            # And append it into the list.
            lips_landmarks.append(face_landmarks[index])
        
        # Convert the lips landmarks list into a numpy array.
        lips_landmarks = np.array(lips_landmarks, np.int32)
        
        # Draw filled lips contours on the copy of the image.
        cv2.drawContours(output_image, contours=[lips_landmarks], contourIdx=-1, 
                         color=(255, 255, 255), thickness=-1)
        
        # Check if the index finger tip is inside the lips contours (outline). 
        if cv2.pointPolygonTest(lips_landmarks,(index_x, index_y), measureDist=False)  == 1:
            
            # Update the selected face part variable to LIPS.
            selected_face_part = 'LIPS'
            
        # Eyes Selection part.
        ####################################################################################################################
        
        # Iterate over the indexes of the right eye ouline.
        for index in right_eye_upper_ids+right_eye_lower_ids:
            
            # Get the landmark at the index we are iterating upon,
            # And append it into the list.
            right_eye_landmarks.append(face_landmarks[index])
        
        # Convert the right eye landmarks list into a numpy array.
        right_eye_landmarks = np.array(right_eye_landmarks, np.int32)
        
        # Draw filled right eye contours on the copy of the image.
        cv2.drawContours(output_image, contours=[right_eye_landmarks], contourIdx=-1, 
                         color=(255, 255, 255), thickness=-1)  
        
        # Check if the index finger tip is inside the right eye contours (outline). 
        if cv2.pointPolygonTest(right_eye_landmarks,(index_x, index_y), measureDist=False)  == 1:
            
            # Update the selected face part variable to EYES.
            selected_face_part = 'EYES'
        
        # Face Selection part.
        ####################################################################################################################
        
        # Iterate over the indexes of the right cheek ouline.
        for index in right_cheek_landmarks_ids:
            
            # Get the landmark at the index we are iterating upon,
            # And append it into the list.
            right_cheek_landmarks.append(face_landmarks[index])
        
        # Convert the right cheek landmarks list into a numpy array.
        right_cheek_landmarks = np.array(right_cheek_landmarks, np.int32)
        
        # Draw filled right cheek contours on the copy of the image.
        cv2.drawContours(output_image, contours=[right_cheek_landmarks], contourIdx=-1, 
                         color=(255, 255, 255), thickness=-1)    
        
        # Check if the index finger tip is inside the right cheek contours (outline). 
        if cv2.pointPolygonTest(right_cheek_landmarks,(index_x, index_y), measureDist=False)  == 1:
            
            # Update the selected face part variable to FACE.
            selected_face_part = 'FACE'
        

        # Eyebrow Selection Part (Challenge)
        ####################################################################################################################
        # Iterate over the indexes of the eyebrow ouline.

        # Right Eyebrow
        for index in right_eyebrow_upper_ids+right_eyebrow_lower_ids:
            
            # Get the landmark at the index we are iterating upon,
            # And append it into the list.
            right_eyebrow_landmarks.append(face_landmarks[index])
        
        # Convert the right cheek landmarks list into a numpy array.
        right_eyebrow_landmarks = np.array(right_eyebrow_landmarks, np.int32)
        
        # Draw filled right cheek contours on the copy of the image.
        cv2.drawContours(output_image, contours=[right_eyebrow_landmarks], contourIdx=-1, 
                         color=(255, 255, 255), thickness=-1)    
        
        # Check if the index finger tip is inside the right cheek contours (outline). 
        if cv2.pointPolygonTest(right_eyebrow_landmarks,(index_x, index_y), measureDist=False)  == 1:
            
            # Update the selected face part variable to Right Eyebrow.
            selected_face_part = 'RIGHT EYEBROW'

        
        # Left Eyebrow
        for index in left_eyebrow_upper_ids+left_eyebrow_lower_ids:
            
            # Get the landmark at the index we are iterating upon,
            # And append it into the list.
            left_eyebrow_landmarks.append(face_landmarks[index])
        
        # Convert the left eyebrow landmarks list into a numpy array.
        left_eyebrow_landmarks = np.array(left_eyebrow_landmarks, np.int32)
        
        # Draw filled right cheek contours on the copy of the image.
        cv2.drawContours(output_image, contours=[left_eyebrow_landmarks], contourIdx=-1, 
                         color=(255, 255, 255), thickness=-1)    
        
        # Check if the index finger tip is inside the right cheek contours (outline). 
        if cv2.pointPolygonTest(left_eyebrow_landmarks,(index_x, index_y), measureDist=False)  == 1:
            
            # Update the selected face part variable to Left Eyebrow.
            selected_face_part = 'LEFT EYEBROW'
        
        ####################################################################################################################


        # Perform weighted addition between the original image and 
        # its copy with the contours drawn to get a transparency effect. 
        output_image = cv2.addWeighted(output_image, 0.35, image, 0.65, 0)
    
    # Return the image with transparent contours drawn, and the selected face part
    return output_image, selected_face_part

Now lets see how this **`selectFacePart()`** function works on a real-time webcam feed. To remove the false positives we will only consider a face part selected when the **`selectFacePart()`** function returns that face part name multiple times.

In [6]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Removed these
#camera_video.set(3,1280)
#camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Face Part Selection', cv2.WINDOW_NORMAL)

# Create a buffer to store the selected face part.
selected_face_part = deque([], maxlen=20)

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Get the height and width of the frame of the webcam video. 
    frame_height, frame_width, _ = frame.shape
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Face landmarks detection.
    frame, face_landmarks = detectFacialLandmarks(frame, face_mesh, draw=False, display=False)
    
    # Check if the Face landmarks in the frame are detected.
    if len(face_landmarks)>0:
        
        # Perform Hands landmarks detection on the frame.
        frame, hands_results = detectHandsLandmarks(frame, hands, draw=True, display=False)
        
         # Check if the hands landmarks in the frame are detected.
        if hands_results.multi_hand_landmarks:
            
            # Perform the Face part selection process utilizing the face and hands landmarks.
            frame, currently_selected_part = selectFacePart(frame, face_landmarks, hands_results)
            
            # Append the Face part selection results into the buffer.
            selected_face_part.append(currently_selected_part)
            
            # Check if the length of the buffer is equal to 20 i.e. the max length.
            if len(selected_face_part) == 20:
                
                # Check if the current maximum face part selection results in the buffer are LIPS.
                if max(set(selected_face_part), key=selected_face_part.count) == 'LIPS':
                    
                    # Write LIPS Selected text on the frame. 
                    cv2.putText(frame, 'LIPS Selected.', (5, int(frame_height-20)),
                                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                    
                # Check if the current maximum face part selection results in the buffer are FACE.
                elif max(set(selected_face_part), key=selected_face_part.count) == 'FACE':
                    
                    # Write FACE Selected text on the frame. 
                    cv2.putText(frame, 'FACE Selected.', (5, int(frame_height-20)),
                                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                
                # Check if the current maximum face part selection results in the buffer are EYES.
                elif max(set(selected_face_part), key=selected_face_part.count) == 'EYES':
                    
                    # Write EYES Selected text on the frame. 
                    cv2.putText(frame, 'EYES Selected.', (5, int(frame_height-20)),
                                 cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

                # Check if the current maximum face part selection results in the buffer are RIGHT EYEBROWS
                elif max(set(selected_face_part), key=selected_face_part.count) == 'RIGHT EYEBROW':
                    # Write EYES Selected text on the frame. 
                    cv2.putText(frame, 'RIGHT EYEBROW.', (5, int(frame_height-20)),
                                 cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                
                # Check if the current maximum face part selection results in the buffer are RIGHT EYEBROWS
                elif max(set(selected_face_part), key=selected_face_part.count) == 'LEFT EYEBROW':
                    # Write EYES Selected text on the frame. 
                    cv2.putText(frame, 'LEFT EYEBROW.', (5, int(frame_height-20)),
                                 cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
                    
    # Display the frame.
    cv2.imshow('Face Part Selection', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()

Nice! working perfectly fine.

##  <font style="color:rgb(34,169,134)">Challenge (Optional)</font>

Inside the **`selectFacePart()`** function created above, try to add the functionality of selecting individual eyebrows, this can be used in the next lesson, where you'll be change the eyebrow colors.

In [7]:
## ADD CODE HERE

# I finished the challenge here, I directly added it to the select face part function.
# I have also recorded myself testing it

# As for my references on the landmarks, here it is:
#       https://github.com/ManuelTS/augmentedFaceMeshIndices/blob/master/Frong_wireframe.jpg
#       https://github.com/ManuelTS/augmentedFaceMeshIndices/blob/master/Right_Eye.jpg
#       https://github.com/ManuelTS/augmentedFaceMeshIndices/blob/master/Left_Eye.jpg

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```
